In [1]:
pip install tqdm

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

#0. Initial Setting

In [3]:
import numpy as np
from scipy.special import logsumexp
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

theta1 = np.array([1,-1.9,-1.8,2]) #RS, FC1,FC2, EC

theta23 = 0.2
agent = 2
za_dim =(agent+1)*(2**agent)
za_old = np.arange(za_dim,dtype = int)

TimePeriod= 200
SamplePath = 300

z_old = np.arange(agent+1,dtype=int) #observation
a1_old = np.arange(2,dtype = int)
a2_old = np.arange(2,dtype = int)


In [4]:
#print(za_old//(agent+1))
print((za_old//(agent+1))//2) #a1
print((za_old//(agent+1))%2)  #a2
print(za_old%(agent+1))       #z

np.hstack([np.hstack([a1_old[i] *2 + a2_old for i in range(2)])[j] *3+z_old for j in range(4)])

[0 0 0 0 0 0 1 1 1 1 1 1]
[0 0 0 1 1 1 0 0 0 1 1 1]
[0 1 2 0 1 2 0 1 2 0 1 2]


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

#1. Generate Dynamic Part and Policy Part

In [5]:
def Dynamic(theta23):
  
  observe = np.zeros([3,3]) #z_t,z_{t+1}
  observe[0,1] = observe[1,0] = observe[1,2]= observe[2,1] = theta23
  observe[0,0] = 1 - observe[0,1] -observe[0,2]
  observe[1,1] = 1 - observe[1,0] -observe[1,2]
  observe[2,2] = 1 - observe[2,0] - observe[2,1]

  return observe
#print(np.kron(np.ones(4),Dynamic(theta23)).T)

def Reward(theta1,ag_i): # 3*2*2, 2,2
  rw = np.zeros([(agent+1)*(2**agent),2,2])#z*a_{t-1},a_{t,1},a_{t,2}
  if ag_i==1:
    rw[:,1,0] = (za_old%(agent+1)) * theta1[0]/((2+0)**2) - theta1[1] - theta1[3] * (1-((za_old//(agent+1))//2))
    rw[:,1,1] = (za_old%(agent+1)) * theta1[0]/((2+1)**2) - theta1[1] - theta1[3] * (1-((za_old//(agent+1))//2))
  elif ag_i==2:
    rw[:,0,1] = (za_old%(agent+1)) * theta1[0]/((2+0)**2) - theta1[2] - theta1[3] * (1-((za_old//(agent+1))%2))
    rw[:,1,1] = (za_old%(agent+1)) * theta1[0]/((2+1)**2) - theta1[2] - theta1[3] * (1-((za_old//(agent+1))%2))    

  return rw
#print(Reward(theta1,2))

def ValueIteration(theta1,theta23,beta = 0.95,
                   gamma = 0.5772,thread_inner = 1e-9,thread_outer = 1e-9,max_iter = 50000): #zt * a_old, a1 or zt * a_old,a2
  observe = Dynamic(theta23)
  observe = np.kron(np.ones(4),observe).T
  rw1,rw2 = Reward(theta1,1), Reward(theta1,2) #z*a_{t-1},a_{t,1},a_{t,2}

  pi1_new = 0.5*np.ones([za_dim,2])#zt*a_old, a1
  pi2_old = pi1_new.copy()

  for ol in range(max_iter):
    pi1_old = pi1_new.copy()
    
    Q1_new = np.zeros([za_dim,2]) #xt,zt,a_old, a1
    Q2_new = np.zeros([za_dim,2])

    for il in range(max_iter):
      Q2_old = Q2_new.copy()
      for a2 in range(2):
        #a1=0
        znew_a10= (0 *2+a2)*3 * np.ones([za_dim,1],dtype = int)+z_old
        #print(znew_a10) 
        V2_a10 = Q2_old[znew_a10,:]
        Q2_tepa10 = rw2[:,0,a2] + beta * np.sum(observe*(gamma+logsumexp(V2_a10,axis=2)),axis=1)

        #a1=1
        znew_a11= (1 *2+a2)*3 * np.ones([za_dim,1],dtype = int)+z_old
        V2_a11 =  Q2_old[znew_a11,:]
        Q2_tepa11 =rw2[:,1,a2] +  beta * np.sum(observe*(gamma+logsumexp(V2_a11,axis=2)),axis=1)
          
        Q2_new[:,a2] = pi1_old[:,0] *Q2_tepa10 + pi1_old[:,1]* Q2_tepa11
      #print(il,np.max(np.abs(((Q2_new-Q2_old)/Q2_old))))
      if np.max(np.abs(((Q2_new-Q2_old)/Q2_old))) <thread_inner: 
        #print('Q2',il,np.max(np.abs((Q2_new-Q2_old)/Q2_old)))
        break

    pi2_new = softmax(Q2_new,axis=1)
    count = np.max(np.abs(((pi2_old-pi2_new)/pi2_old))) 

    pi2_old = pi2_new.copy()
    for il in range(max_iter):
      Q1_old = Q1_new.copy()
      for a1 in range(2):
        #a2 = 0
        znew_a20= (a1 *2+0)*3 * np.ones([za_dim,1],dtype = int)+z_old
        V1_a20 =  Q1_old[znew_a20,:]
        Q1_tepa20 =rw1[:,a1,0] + beta * np.sum(observe*(gamma+logsumexp(V1_a20,axis=2)),axis=1)

        #a2=1
        znew_a21= (a1 *2+1)*3 * np.ones([za_dim,1],dtype = int)+z_old
        V1_a21 =  Q1_old[znew_a21,:]
        Q1_tepa21 = rw1[:,a1,1] + beta * np.sum(observe*(gamma+logsumexp(V1_a21,axis=2)),axis=1)
          
        Q1_new[:,a1] = pi2_old[:,0] *Q1_tepa20 + pi2_old[:,1] * Q1_tepa21
      #print(il,np.max(np.abs(((Q1_new-Q1_old)/Q1_old))))
      if np.max(np.abs(((Q1_new-Q1_old)/Q1_old))) <thread_inner:
        #print('Q1',il,np.max(np.abs((Q1_new-Q1_old)/Q1_old)))
        break

    pi1_new = softmax(Q1_new,axis=1)
    
    #print('pi1 pi2',ol,np.max(np.abs(((pi1_old-pi1_new)/pi1_old))),count)
    if np.max(np.abs(((pi1_old-pi1_new)/pi1_old))) <thread_outer and count<thread_outer:
      #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
      break
  return Q1_new,Q2_new,pi1_new,pi2_new

Q1,Q2,pi1,pi2 = ValueIteration(theta1,theta23)

In [6]:
np.random.seed(3)
ms_mat = np.random.random(size=5)
observe= Dynamic(theta23)
haha = 2*np.ones(5,dtype=int)
print(ms_mat,np.int_(observe[haha,0]<=ms_mat) +np.int_(observe[haha,0]+observe[haha,1]<=ms_mat))

[0.5507979  0.70814782 0.29090474 0.51082761 0.89294695] [2 2 2 2 2]


#2. Data Generation Process

In [7]:
##################################Global Parameter#################################
# theta1 = np.array([3,1.9,2]) #RS, FC, EC
# theta23 = 0.2
# agent = 2
# za_dim =(agent+1)*(2**agent)
# za_old = np.arange(z_dim,dtype = int)
# TimePeriod= 150
# SamplePath = 300
# num_discrete = 100
# z_old = np.arange(agent+1,dtype=int) #observation
# a1_old = np.arange(2,dtype = int)
# a2_old = np.arange(2,dtype = int)
###################################################################################
from tqdm import tqdm 

def DataGeneration(theta1 = theta1,theta23=theta23,T=TimePeriod,N=SamplePath):
  # Assume start from good condition and highest market size(2), agents entry market(1) at time 0

  market_size = 2 * np.ones([T,N],dtype = int) #observation state
  action1 = np.ones([T,N],dtype = int)   #agent1's action
  action2 = np.ones([T,N],dtype = int)   #agent2's action
      

  observe = Dynamic(theta23)

  _,_,pi1,pi2 = ValueIteration(theta1,theta23)
  for t in tqdm(range(1,T)):
    # generate new market size
    ms_mat = np.random.random(size=N)
    pt = observe[market_size[t-1,:],:]
    market_size[t,:] = np.int_(pt[:,0]<=ms_mat) +np.int_(pt[:,0]+pt[:,1]<=ms_mat)


    # generate new action for both agents
    za = (action1[t-1,:] *2 + action2[t-1,:])*3+market_size[t,:]
    action_mat1 = np.random.random(size=N)
    p1 = pi1[za,0]
    action1[t,:] = np.int_(p1<=action_mat1)
    
    action_mat2 = np.random.random(size=N)
    p2 = pi2[za,0]
    action2[t,:] = np.int_(p2<=action_mat2)
    
  return market_size, action1,action2


market_size, action1,action2 = DataGeneration()
#haha = SigmaLambda()

100%|██████████| 199/199 [00:00<00:00, 15556.75it/s]


#3.1 Recover Dynamic Part

In [ ]:
theta23

0.2

In [ ]:
from scipy.optimize import minimize
ini_x0 = 0.001*theta23
print('start from:',ini_x0 )

def func1(theta23,market_size=market_size, action1=action1,action2 = action2,T=TimePeriod):
  observe = Dynamic(theta23)
  #print(observe[market_size[0:-1,:],market_size[1:T,:]])
  res = -np.sum(np.log(observe[market_size[0:-1,:],market_size[1:T,:]]))
  #print(theta23,res)
  return res

res1 = minimize(func1,ini_x0, bounds=[[0,0.499999]])
#trans_est = np.vstack([res1.x,1-res1.x]).T.reshape([2,2,2,2])
print(res1)

start from: 0.0002
      fun: 38953.14401538699
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 22
      nit: 6
   status: 0
  success: True
        x: array([0.20096792])


#3.2 Recover Reward Part

In [ ]:
theta1.reshape(-1)

array([ 1. , -1.9, -1.8,  2. ])

In [ ]:
ini_x0 = 0.85*theta1.reshape(-1)
print('start from:',ini_x0 )

def func2(theta1,theta23=res1.x,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod):
  #theta1 = theta1.reshape([2,2,2])
  _,_,pi1,pi2 = ValueIteration(theta1,theta23)

  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]

  p1 = pi1[za,action1[1:T,:]]
  p2 = pi2[za,action2[1:T,:]]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res

res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,(1e-6,100)])#method='Nelder-Mead')
print(res2)

start from: [ 0.85  -1.615 -1.53   1.7  ]
[ 0.85  -1.615 -1.53   1.7  ] 14792.378750107295
[ 0.85000001 -1.615      -1.53        1.7       ] 14792.378747028602
[ 0.85       -1.61499999 -1.53        1.7       ] 14792.378762857146
[ 0.85       -1.615      -1.52999999  1.7       ] 14792.378764575364
[ 0.85       -1.615      -1.53        1.70000001] 14792.378730432454
[ 100. -100. -100.  100.] 605197.777777778
[ 100.00000001 -100.         -100.          100.        ] 605197.7777810852
[ 100.          -99.99999999 -100.          100.        ] 605197.7777625578
[ 100.         -100.          -99.99999999  100.        ] 605197.7777610379
[ 100.         -100.         -100.          100.00000001] 605197.7778030303
[ 13.31443192 -13.98326157 -13.90894717  14.05757598] 84468.4109090738
[ 13.31443193 -13.98326157 -13.90894717  14.05757598] 84468.4109123816
[ 13.31443192 -13.98326156 -13.90894717  14.05757598] 84468.41089385381
[ 13.31443192 -13.98326157 -13.90894716  14.05757598] 84468.41089233375
